# Bayesian Statistics Workshop - Applied Portion
### Hosted by: Eric Zhu | [ezhu.build](https://ezhu.build)
---
The workshop slides will be available on my [Github](https://github.com/GreatArcStudios), which you should be able to find in the same directory as this notebook.

## Aims
- Creating two Bayesian models
- Understanding some pros/cons of Bayesian methods
- Some basic prior setting
- Evaluating Bayesian model performance
- Understanding how to create predictions through the posterior predictive distribution

## Breakdown
- Creating a multiple imputation model (a model that generates a distribution for the "population" and uses that distribution to generate a distribution for our quantity of interest) with random effects.
- The data, while simulated, is provided in the context that a large university is trying to analyse student marks in a large first year class (think Convocation (Con) Hall classes at UofT). In this scenario, the University asked the students to voluntarily participate in a pre-study exam to examine the effects of studying. This pre-study exam is equivalent in skill & difficulty wise to the exam everyone gets. Additionally, the University has access to every student's high school GPA, lecture section, and amount of time studied (perhaps they need to use a VPN to connect to the student portal).
- The student GPAs are on a 4.0 scale, and generally exam scores are within the range of 0-110 due to bonus marks. Ideally, we would use a truncated normal distribution, but for the sake of time we will not cover it.

## Credits

While I  created the materials/scenario for this workshop, I must also credit [Professor Dan Simpson](https://dpsimpson.github.io/) for the inspiration. This workshop was based off of a similar scenario from Homework 2 of the Winter 2021 offering of [STA365](https://www.statistics.utoronto.ca/sites/www.statistics.utoronto.ca/files/STA365H1%20sylabus2021%20%283%29.pdf) at the University of Toronto (UofT). We are not explicitly covering post stratification or truncated distributions in the workshop for the sake of time/accessibility. Additionally, explanations of random effect models generally follow those given by [Professor Liza Bolton](https://www.dataembassy.co.nz/) during the Winter 2021 offering of [STA303/1002](https://www.statistics.utoronto.ca/sites/www.statistics.utoronto.ca/files/STA303-1002_syllabus_W21.pdf).

In [ ]:
# packages we need in this workshop
import math
import numpy as np
import pandas as pd
import cmdstanpy
import arviz as az
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
plt.rcParams["figure.figsize"] = (10,8)
sns.set_style('whitegrid')

# Part 0: Generating the Data
In this workshop, we'll be modelling some generated data. While this means we have a population distribution (the distribution from which we generate from), you'll get to see how Bayesian methods perform in (assumed) frequentist settings, which is pretty cool!

For later: we will be leaving half of the data in one column out in the data I provide since we will be creating a model to predict that data.

### Important: don't look at this section until the end

In [ ]:
# init the numpy array to 0s (dataset will be 4-dim)
samples = 5000
data = np.zeros((samples, 5))
rng = np.random.default_rng(seed=123)
# certain lecture sections have on average higher scores
section_dict = {"LEC0101": 5, "LEC0202": 0, "LEC0303": -1.25, "LEC0404": 2}
lecture_sections = rng.choice(["LEC0101", "LEC0202", "LEC0303", "LEC0404"], samples)
entrance_gpa = rng.normal(3.85, 0.05, samples)
prestudy_scores = []
study_times = []
poststudy_scores = []
def generate_data():
    for sample in range(samples):
        lec_section = lecture_sections[sample]
        prestudy_scores.append(rng.normal(57+0.45*entrance_gpa[sample]+section_dict[lec_section], 10, 1))
        study_times.append(rng.normal(10+section_dict[lec_section], 2, 1))
        poststudy_scores.append(rng.normal(58+0.35*entrance_gpa[sample] + 0.5*prestudy_scores[sample] + study_times[sample]*1.2+section_dict[lec_section], 3.25, 1))

generate_data_vectorized = np.vectorize(generate_data)

generate_data_vectorized()
data[:,1] = prestudy_scores
data[:,2] = entrance_gpa
data[:,3] = study_times
data[:,4] = poststudy_scores
data_df = pd.DataFrame(data, columns=["Lecture_Section", "PreStudy_Scores","Entrance_GPA", "Study_Times", "PostStudy_Scores"])
data_df = data_df.round(2)
data_df.iloc[:,0] = lecture_sections
fully_labelled_data = data_df.iloc[:1000]
#print(data_df)
fully_labelled_data.to_csv("full_labelled_workshop_data.csv")
missing_data = data_df.iloc[:1000]
missing_data = missing_data[['Lecture_Section', 'Entrance_GPA', 'Study_Times']]
poststudy_scores_keep = pd.Series(data_df.PostStudy_Scores[:500])
prestudy_scores_keep = pd.Series(data_df.PreStudy_Scores[:500])
missing_data.insert(1, "PreStudy_Scores", prestudy_scores_keep)
missing_data.insert(1, "PostStudy_Scores", poststudy_scores_keep)
missing_data.to_csv("missing_data_workshop_data.csv")

# Part 1: Creating a simple model
We will use this model as the model that creates a population distribution for the missing data, i.e., we will predict the missing data.

To do so, we will use a random effects model, specifically a random effect for $\texttt{lecture_section}$:

\begin{aligned}
\texttt{PreStudy_Scores}_i &= \beta_0 + \beta_1 \cdot \texttt{Entrance_GPA}_i + U_{ij} + \epsilon \\
\beta_0 &\sim N(\mu_{\beta_0}, \tau_{\beta_0}) \\
\beta_1 &\sim N(\mu_{\beta_1}, \tau_{\beta_1}) \\
U &\sim N(0, \tau_U) \\
\tau_{U} &\sim N_+(\mu_{\tau_U}, \tau_{\tau_U}) \\
\sigma &\sim N_+(0, \tau_{\sigma})
\end{aligned}

Note that $U_{ij}$ is the random effect for the $j^{th}$ lecture section that corresponds to the $i^{th}$ individual. Additionally, with a normally distributed error term recall that for models like linear regression, we may represent the model as the likelihood (with $X$ as the $n$ by $p$ design matrix and $w$ as $p$-dim vector of weights): $ y \sim N(Xw, \sigma^2)$.

In $\texttt{lme4}$ style syntax, we may write:

$\texttt{PreStudy_Scores}_i  = \texttt{Entrance_GPA}_i  + (1~|~\texttt{Lecture_Section}_i )$

So we will need 4 priors: $\beta_0, \beta_1, \tau_U, \sigma^2$.

For the first prior, the prior for the intercept, you know that on average students score around a 55% without studying. Although some students are quite good (or perhaps lucky) and score up to 20% higher without studying

As for the prior on $\beta_1$, we know that a student's High School or Secondary Education GPA has a mild effect on their scores before any studying. Perhaps something like a factor of 0.5, although since we know the effect of GPA isn't that large, we don't expect the effect to be sensibly more than twice the average effect (the centre of your prior distribution).

Then for the standard deviation of the random intercept, we know that the intercept can vary in extreme cases by up to 5, i.e., sometimes the lecture section may increase a student's performance by 5 percent! So then, you should expect the lecture section to affect student performance by at least a couple of percent on average, perhaps $\mu_{\tau_U} \approx 2$.

Finally, for $\sigma^2$, the variance of the data, we expect that the distribution is centred around 0. In other words, we can expect our model to be centred around our mean. But the standard deviation of $\sigma^2$ is then rather large, perhaps we could see students who perform up to 20 or 30 percent better holding all other factors constant!

### Remember to use half normal distributions for standard derivation parameters since normal distribution standard deviations are non-negative.

## Code!

In [ ]:
# read the data
workshop_data_missing = pd.read_csv("missing_data_workshop_data.csv", index_col=0)
#convert to numerical factor levels
workshop_data_missing['LecSec_Numeric'] = [code+1 for code in pd.Categorical(workshop_data_missing.Lecture_Section).codes]
workshop_data_missing

labelled_data = workshop_data_missing[:500]# get the first two hundred rows and all columns

In [ ]:
# Quick EDA, NOT used for determining priors!

# Pre-study scores are roughly normally distributed, as expected
prestudy_plot = plot_distribution(workshop_data_missing.dropna().PreStudy_Scores)

In [ ]:
# Post-study scores
poststudy_plot = plot_distribution(workshop_data_missing.PostStudy_Scores)

In [ ]:
# number of lecture sections
pd.value_counts(labelled_data.Lecture_Section)
lecsec_plot = sns.histplot(labelled_data.LecSec_Numeric, discrete=True, color="#03396c", edgecolor="#011f4b", alpha=1)

In [ ]:
# compile the model
prestudy_model = cmdstanpy.CmdStanModel(stan_file="workshop_prestudy.stan")

In [ ]:
# create the data lists for Stan
data_list_prestudy = {'N':500,
                      'prestudy_scores': list(labelled_data.PreStudy_Scores),
                      'J_lecsec': len(labelled_data.Lecture_Section.unique()),
                      'lec_sec': list(labelled_data.LecSec_Numeric),
                      'entrance_gpa': list(labelled_data.Entrance_GPA),
                      'mu_sigma' : 0,
                      'tau_sigma': 25,
                      'mu_beta1': 0.5,
                      'tau_beta1': 0.15,
                      'mu_intercept': TODO,
                      'tau_intercept': TODO,
                      'mu_tau_lecsec': 2,
                      'tau_tau_lecsec': 1.25,
                      'only_prior': 1
                      }
# fit the model
prior_prestudy = prestudy_model.sample(data=data_list_prestudy,
                                       parallel_chains=4,
                                       iter_sampling=1000,
                                       iter_warmup=1000,
                                       refresh=500,
                                       show_progress=True)

In [ ]:
data_list_prestudy['only_prior'] = 0
posterior_prestudy = prestudy_model.sample(data=data_list_prestudy,
                                       parallel_chains=4,
                                       iter_sampling=1000,
                                       iter_warmup=1000,
                                       refresh=500)

In [ ]:
prestudy_az = az.from_cmdstanpy(
    posterior=posterior_prestudy,
    posterior_predictive="y_pred",
    observed_data={'y_pred': list(data_list_prestudy['prestudy_scores'])},
    log_likelihood="log_lik",
    prior = prior_prestudy,
    prior_predictive="y_pred"
)

## Model checking/evaluation

In [ ]:
# prior draws
prior_prestudy_draws = prior_prestudy.draws_pd()
prior_prestudy_draws

# plot posterior pred. KDE estimates
# subset columns we want to plot
prestudy_prior_pred_subset = prior_prestudy_draws.filter(like="y_pred").sample(n=9,axis='columns')
plot_pred_distributions(prestudy_prior_pred_subset)

In [ ]:
# posterior draws
posterior_prestudy_draws = posterior_prestudy.draws_pd()
posterior_prestudy_draws

# plotting, we will perform posterior checks and predictive checks

# plot the comparison plots
# beta_0 comparison plot
plot_prior_posterior(prior_prestudy_draws.beta_0, posterior_prestudy_draws.beta_0, parameter_name="beta_0")
# beta_1 comparison plot
plot_prior_posterior(prior_prestudy_draws.beta_1, posterior_prestudy_draws.beta_1, parameter_name="beta_1")
# random effect st dev. comparison plot
plot_prior_posterior(prior_prestudy_draws.tau_lecsec, posterior_prestudy_draws.tau_lecsec, parameter_name="tau_lecsec")

# plot the posterior pred. distribution ppc
az.plot_ppc(prestudy_az)

# plot posterior pred. KDE estimates
# subset columns we want to plot
prestudy_posterior_pred_subset = posterior_prestudy_draws.filter(like="y_pred").sample(n=9,axis='columns')
plot_pred_distributions(prestudy_posterior_pred_subset)

# plot the PSIS plot
az.plot_khat(az.loo(prestudy_az, pointwise=True))

# summaries
az.summary(prestudy_az)

# Part 2: Creating the complex model

Now we use the predicted population distribution to construct the distribution of post-study scores. We must use many samples of each lecture section to accurately predict the post-study score. The core idea here is that we must incorporate all of our information about pre-study scores into our distribution for the post-study scores; in other words, we marginalize over the pre-intervention scores.

Then, our model will be:

\begin{aligned}
\texttt{PostStudy_Scores}_i &= \beta_0 + \beta_1 \cdot \texttt{Entrance_GPA}_i + \\
&\beta_2 \cdot \texttt{PreStudy_Scores}_i + \beta_3 \cdot \texttt{Study_Times}_i + U_{ij} + \epsilon \\
\beta_0 &\sim N(\mu_{\beta_0}, \tau_{\beta_0}) \\
\beta_1 &\sim N(\mu_{\beta_1}, \tau_{\beta_1}) \\
\beta_2 &\sim N(\mu_{\beta_2}, \tau_{\beta_2}) \\
\beta_3 &\sim N(\mu_{\beta_3}, \tau_{\beta_3}) \\
U &\sim N(0, \tau_U) \\
\tau_{U} &\sim N_+(\mu_{\tau_U}, \tau_{\tau_U})\\
\sigma &\sim N_+(0, \tau_{\sigma})
\end{aligned}

Note that $U_{ij}$ is the random effect for the $j^{th}$ lecture section that corresponds to the $i^{th}$ individual. Again, with a normally distributed error term recall that for models like linear regression, we may represent the model as the likelihood (with $X$ as the $n$ by $p$ design matrix and $w$ as $p$-dim vector of weights): $ y \sim N(Xw, \sigma^2)$.

In $\texttt{lme4}$ style syntax, we may write:

$\texttt{PostStudy_Scores}_i  = \texttt{PreStudy_Scores}_i + \texttt{Entrance_GPA}_i  + (1~|~\texttt{Lecture_Section}_i)$

So we will need 6 priors: $\beta_0, \beta_1, \beta_2, \beta_3, \tau_U, \sigma^2$.

For the first prior, the prior for the intercept, you know that on average students score around a 55% without studying.

As for the prior on $\beta_1$, we know that a student's High School or Secondary Education GPA has an effect on their scores before any studying. Perhaps something like a factor of 0.3, although since we know the effect of GPA isn't that large, we don't expect the effect to be sensibly more than twice the average effect (the centre of your prior distribution) for almost all students.

For the prestudy scores, we know that a student's pre-study scores can be used a useful predictor of post-study scores, e.g., someone who scores highly on the pre-study exam is likely to score highly again. So then, we expect a mild effect perhaps like a factor of 0.4, and it should have a similar standard deviation to $\beta_1$.

Additionally, we know that studying ($\beta_3$) has an enormous effect on post-study scores. Perhaps in previous years, the University measured the effect to be around 1.2, with generally some small variation in the effect size, i.e., $\pm 0.2$.

Then for the variance of the random intercept, we know that the intercept can vary in extreme cases by up to 5, i.e., sometimes the lecture section may increase a student's performance by 5 percent! So then, you should expect the lecture section to affect student performance by at least a couple of percent on average, perhaps $\mu_{\tau_U} \approx 2$.

Finally, for $\sigma^2$, the variance of the data, we expect that the distribution is centred around 0. In other words, we can expect our model to have no error. But the variance of $\sigma^2$ is then rather large, perhaps we could see students who perform up to 20 percent better despite all other factors constant!


## Code!

In [ ]:
# compile the model
poststudy_model = cmdstanpy.CmdStanModel(stan_file="workshop_poststudy.stan")

In [ ]:
# create the data lists for Stan
data_list_poststudy = {'N':500,
                      'prestudy_scores': list(labelled_data.PreStudy_Scores),
                      'J_lecsec': len(labelled_data.Lecture_Section.unique()),
                      'lec_sec': list(labelled_data.LecSec_Numeric),
                      'entrance_gpa': list(labelled_data.Entrance_GPA),
                      'poststudy_scores': list(labelled_data.PostStudy_Scores),
                      'study_times': list(labelled_data.Study_Times),
                      'mu_sigma' : TODO,
                      'tau_sigma': TODO,
                      'mu_beta1': TODO,
                      'tau_beta1': TODO,
                      'mu_beta2': 0.5,
                      'tau_beta2': 0.15,
                      'mu_beta3': TODO,
                      'tau_beta3': TODO,
                      'mu_intercept': 55,
                      'tau_intercept': 10,
                      'mu_tau_lecsec': 2,
                      'tau_tau_lecsec': 1.25,
                      'only_prior': 1
                      }
# fit the model
prior_poststudy = poststudy_model.sample(data=data_list_poststudy,
                                       parallel_chains=4,
                                       iter_sampling=1000,
                                       iter_warmup=1000,
                                       refresh=500)

In [ ]:
data_list_poststudy['only_prior'] = 0
posterior_poststudy = poststudy_model.sample(data=data_list_poststudy,
                                       parallel_chains=4,
                                       iter_sampling=1000,
                                       iter_warmup=1000,
                                       refresh=500)

In [ ]:
poststudy_az = az.from_cmdstanpy(
    posterior=posterior_poststudy,
    posterior_predictive='y_pred',
    observed_data={'y_pred': list(data_list_poststudy['poststudy_scores'])},
    log_likelihood="log_lik",
    prior = prior_poststudy,
    prior_predictive= 'y_pred'
)

## Model checking/evaluation

In [ ]:
# prior draws
prior_poststudy_draws = prior_poststudy.draws_pd()
prior_poststudy_draws

# plot posterior pred. KDE estimates
# subset columns we want to plot
poststudy_prior_pred_subset = prior_poststudy_draws.filter(like="mu").sample(n=9,axis='columns')
plot_pred_distributions(poststudy_prior_pred_subset)

In [ ]:
# posterior draws
posterior_poststudy_draws = posterior_poststudy.draws_pd()
posterior_poststudy_draws

# plotting, we will perform posterior checks and predictive checks

# plot the comparison plots
# beta_0 comparison plot
plot_prior_posterior(prior_poststudy_draws.beta_0, posterior_poststudy_draws.beta_0, parameter_name="beta_0")
# beta_1 comparison plot
plot_prior_posterior(prior_poststudy_draws.beta_1, posterior_poststudy_draws.beta_1, parameter_name="beta_1")
# beta_1 comparison plot
plot_prior_posterior(prior_poststudy_draws.beta_2, posterior_poststudy_draws.beta_2, parameter_name="beta_2")
plot_prior_posterior(prior_poststudy_draws.beta_3, posterior_poststudy_draws.beta_3, parameter_name="beta_3")
# random effect st dev. comparison plot
plot_prior_posterior(prior_poststudy_draws.tau_lecsec, posterior_poststudy_draws.tau_lecsec, parameter_name="tau_lecsec")

# plot the posterior pred. distribution ppc
az.plot_ppc(poststudy_az)

# plot posterior pred. KDE estimates
# subset columns we want to plot
poststudy_posterior_pred_subset = posterior_poststudy_draws.filter(like="y_pred").sample(n=9,axis='columns')
plot_pred_distributions(poststudy_posterior_pred_subset)

# plot the PSIS plot
az.plot_khat(az.loo(poststudy_az, pointwise=True))

# summaries
az.summary(poststudy_az)

# Evaluate how studying went
We will first need to predict the population distribution for pre-study scores and then use those predictions to predict the distribution for post-study scores. By doing so we are able to consider all of the entrance GPAs, study times, lecture sections, and pre-study scores, i.e., we are able to marginalize out everything and obtain the distribution for just post-study scores.

## Create the predictions
We create these outside of Stan since it is easier to do so. It would be possible to do so, but it would be less clean.

In [ ]:
# first subset the draws dataframes and transpose them so that we have each row corresponding to an individual
posterior_prestudy_ppd = posterior_prestudy_draws.filter(like='y_pred').T
posterior_poststudy_ppd = posterior_poststudy_draws.filter(like='y_pred').T

In [ ]:
def create_pred_distribution(data_df, population_df, impute_var ='prestudy_scores', predictor_list = [], impute_df = None,  n_samples=20):
    """
    We want to predict the population distribution by drawing n_samples from the PPD.
    Choose 20 samples by default because of time constraints and works reasonably well
    :param data_df: the posterior pred. distribution df
    :param population_df: the population df, i.e., the dataframe constructed from given data
    :param n_samples: the number of samples to draw from the PPD
    :param predictor_list: the list of predictors we use to create predictions
    :param impute_var: the variable we will impute
    :param impute_df: the datafram containing the data we impute from
    :return: a dataframe of the predicted population distribution
    """
    popn_distribution = pd.DataFrame()
    for index, row in population_df.iterrows():
        lecture_section = row.LecSec_Numeric
        matching_sections = data_df.filter(like=f'u_lecsec[{lecture_section}]')
        prediction = pd.Series(np.zeros(shape=(n_samples)))
        for predictor, var_name in predictor_list:
            # get the vector of posterior weights for each coeff
            if 'u_lecsec' in predictor:
                posterior_weights = matching_sections.sample(n_samples).iloc[:, 0]
            elif predictor == impute_var:
                ppd_prediction_draws = impute_df.filter(like='y_pred').T
                posterior_weights = data_df[predictor].sample(n_samples) * ppd_prediction_draws.sample(n_samples)
            else:
                posterior_weights = data_df[predictor].sample(n_samples)
                if predictor != 'beta_0':
                    posterior_weights = posterior_weights * population_df[var_name].iloc[index]
            # generate the prediction
            prediction = prediction + posterior_weights.reset_index(drop=True)
        popn_distribution = popn_distribution.append(prediction, ignore_index=True)
    return popn_distribution

In [ ]:
# compute the pre-study population distribution
prestudy_popn = create_pred_distribution(posterior_prestudy_draws,
                                         workshop_data_missing,
                                         predictor_list=[('beta_0',''), ('beta_1','Entrance_GPA'), ('u_lecsec','')])

plot_distribution(prestudy_popn)

In [ ]:
# compute the post-study population distribution
poststudy_popn = create_pred_distribution(posterior_poststudy_draws,
                                         workshop_data_missing,
                                         predictor_list=[('beta_0',''),
                                                         ('beta_1','Entrance_GPA'),
                                                         ('beta_2', 'PreStudy_Scores'),
                                                         ('beta_3', 'Study_Times'),
                                                         ('u_lecsec','')])

plot_distribution(poststudy_popn)


In [ ]:
# compute the difference
study_diff = poststudy_popn - prestudy_popn
plot_distribution(study_diff)

## Now for the comparison to the simulated data!

In [ ]:
full_data = pd.read_csv("full_labelled_workshop_data.csv", index_col=0)
plot_distribution(full_data.PreStudy_Scores)
plot_distribution(full_data.PostStudy_Scores)
plot_distribution(full_data.PostStudy_Scores - full_data.PreStudy_Scores)

# Util functions

In [ ]:
def plot_prior_posterior(prior_data, posterior_data, parameter_name=""):
    sns.distplot(prior_data, color="#03396c", hist_kws=dict(edgecolor="#011f4b", alpha=1), kde=False) #prior
    sns.distplot(posterior_data, color="#d1e1ec", hist_kws=dict(edgecolor="#b3cde0", alpha=1), kde=False)# posterior
    plt.title(f'{parameter_name} prior (dark), posterior (light) comparison')
    plt.xlabel('scores')
    plt.ylabel('density')
    plt.figure()

def plot_distribution(data, parameter_name=""):
    sns.distplot(data, color="#03396c", hist_kws=dict(edgecolor="#011f4b", alpha=1), kde=False)# posterior
    plt.title(f'{parameter_name}')
    plt.xlabel('scores')
    plt.ylabel('density')
    plt.figure()

def plot_pred_distributions(pred_distn_df):
    grid_size = math.sqrt(len(pred_distn_df.columns))
    plt.tight_layout()
    for i in range(len(pred_distn_df.columns)):
        data = pred_distn_df.iloc[:, i] # column as a series
        plt.subplot(grid_size, grid_size, i+1)
        sns.distplot(data, color="#03396c", hist_kws=dict(edgecolor="#011f4b", alpha=1), kde=False)# posterior
        plt.title(f'{data.name}')
        plt.xlabel('scores')
        plt.ylabel('density')
    plt.figure()